# 查询操作
- 查询一般就是使用filter、execute以及get三个方法来实现，可以在调用这些方法的时候传递不同的参数来实现查询需求。在ORM层面，这些查询条件都是使用样式为：field+__+conditions来使用的

### 查找方法的解释
- students = Students.objects.annotate(grade_avg=Avg("scores__grade")).filter(grade_avg__gte=80).values("name", "grade_avg")
- 注意：这里的scores指的是外键名，如果在设置外键时设置了related_name，则为related_name的值；要用双下划线，表示从子表中查找相应的属性值
    


# 查询操作常和聚合函数搭配使用

# 查询条件
- 查看原生SQL语句：
    - 方法一：querySet.query,仅限querySet才有query这个属性
            article = Article.objects.filter(id=1)  
            print(article.query)
            #get方法得到就不能用query，get返回的是一个ORM模型
    - 方法二：from django.db import connection, result = connection.queries
        - 该方法得到的原生语句是一个列表，包含很多语句
            
    

## exact
- 使用：字段名__exact= value，exact可以省略，直接写为  字段名=value
- 精确的提供条件，如果提供的是一个None，那么在SQL层面就被解释为Null，即字段名__exact=None为Null
- 注：在Windows操作系统上，MySQL的排序规则（即collation）无论是什么，查找的字符大小写不敏感。但在Linux系统上，MySQL的排序规则若是utf8_bin就是大小写敏感的

## iexact
- 和exact很类似，但是exact是用等号，iexact是用Like，大小写不敏感，无论是在Linux还是Windows中

## contains
- 判断某个字符串是否在指定的字符串中，大小写敏感。即原生SQL语句会使用like binary

## icontains
- 大小写不敏感，原生SQL语句使用like。两个%号表示最字前面或后面可以有任意个字符
        SELECT `a` FROM `anthe` WHERE `title` LIKE %最%
        
## in
- 提取那些给定的field的值是否在给定的容器中，容器是一个可迭代对象，包括QuerySet对象
    - id__in([1,3,4]):表示查找id为1,3,4中的一个或多个
    - 当含有外键时，还可以通过子表的字段来判断是否在某个集合中
        - 比如：article_id_in，其中article是子表名小写而不是article_set，表示查找article的id为1,3,4中的一个或多个
        
## gt，gte，lt，lte
- 代表的是大于，大于等于，小于，小于等于

## startswith
- 判断某个字段的值是否是以某个值开始的，大小写敏感

## istartwith
- 大小写不敏感

## endswith
- 判断某个字段的值是否以某个值结束，iendswith

## range
- 判断某个field的值是否在给定的区域间，如果field是时间类型，要先将时间转换为aware时间类型
- 字段名__range(开始值，结束值),注意不包含最后一个值
- from django.utils.timezone import makr_aware,aware(time)可以将一个时间变为aware时间

## date
- 针对某些date或者datetime类型的字段，可以指定date的范围，并且该过滤还可以使用链式调用
- 注：若在settings.py文件中时区设置为了本地时区，直接进行条件过滤的话得不到想要的结果
    - Magazines.objects.filter(release_time__date=date(year=2020, month=11, day=13))
    - 原生语句为：SELECT xx FROM xx WHERE DATE(CONVERT_TZ(xx.xx, 'UTC', 'Asia/Shanghai')) = 2020-11-13。CONVERT_TZ表示转换为当前时区，但是默认情况下MySQL的表中是没有存储时区相关的信息的，因此我们需要下载时区表的文件。然后添加到MySQL的配置路径中。若是Windows系统，那么在http://dev.mysql.com/downloads/timezones.html 下载timezone_2018d_posix.zip - POSIX standard。然后将下载的所有文件拷贝到\Data\mysql中，如果提示文件名重复，覆盖即可。如果用的是Linux或者mac系统，在命令行中执行以下命令：mysql_tzinfo_to_sql /usr/share/zoneinfo | mysql -D mysql -u root -p，然后输入密码，从系统中加载时区文件更新到MySQL中

## year,month,day,
- 根据年份，月份，号数查找

## week_day
- 根据星期几查找，注意1-7表示星期日到星期六

## time
- 根据时间进行查找，时分秒。秒数查找根据range进行查找

## isnull
- 根据是否为空进行查找
- total_grade__isnull=False

## regex和iregex
- 正则表达式进行查找

## 关联表查询
- 如果一个表引用了另外一个表，还可以通过子表的字段来判断是否在某个集合中
    - 比如：parent_object = Parent.objects.filter(son_name__字段名.contains=''),son_name是子表的小写名，如果使用了related_name参数，则替换成该参数即可